<a href="https://colab.research.google.com/github/mpgl/dendrify-paper/blob/main/Figure_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Install Brian 2
!pip install brian2 --quiet

     |████████████████████████████████| 1.5 MB 5.5 MB/s 


In [ ]:
# @title Imports and setup
import brian2 as b
from brian2.units import (ms, um, pA, nS, uS, ohm, cm, mV, uF, mvolt) # Generally a better practise than * import
from dendrify import Soma, Dendrite, NeuronModel
import matplotlib.pyplot as plt
b.prefs.codegen.target = 'numpy' # Improves performance significantly here

ModuleNotFoundError: ignored

In [ ]:
# @title Figure settings
params = {"font.family": "Arial",
          "legend.fontsize": 10,
          "legend.handlelength": 1.5,
          "legend.edgecolor": 'inherit',
          "legend.columnspacing": 0.8,
          "legend.handletextpad": 0.5,
          "axes.labelsize": 10,
          "axes.titlesize": 11, 
          "axes.spines.right": False,
          "axes.spines.top": False,
          "axes.edgecolor": "#d3d3d3",
          "xtick.color": '#d3d3d3',
          "xtick.labelcolor": 'black',
          "xtick.labelsize": 10,
          "ytick.color": '#d3d3d3',
          "ytick.labelcolor": 'black',
          "ytick.labelsize": 10,
          'mathtext.default': 'regular',
          'lines.markersize': 3,
          'lines.linewidth': 1.25,
          'grid.linestyle': ":",
          'grid.color': "#d3d3d3",
          'text.antialiased': True,
          'lines.antialiased': True,
          'figure.dpi': 400
          }
          
graphite = "#4B4E53"
blue = '#005c94ff'
green = '#338000ff'
orange = '#ff6600ff'
notred = '#aa0044ff'
dsblue = '#0BA8E6'
light_gray = "#b5b5b5"

b.rcParams.update(params)
%matplotlib inline

***Let's first build a basic 3-compartment model with passive dendrites:***

In [ ]:
# create soma
soma = Soma('soma', model='leakyIF', length=25*um, diameter=25*um)

# create apical dendrite
apical = Dendrite('apical', length=250*um, diameter=2*um)

# create basal dendrite
basal = Dendrite('basal', length=150*um, diameter=2*um)

# add noise
apical.noise(tau=20*ms, sigma=3*pA, mean=0*pA)
basal.noise(tau=20*ms, sigma=3*pA, mean=0*pA)

# add synapses
apical.synapse('AMPA', pre='cortex', g=1*nS, t_decay=2*ms)
apical.synapse('NMDA', pre='cortex', g=1*nS, t_decay=60*ms)

# merge compartments into a  neuron model and set its properties
edges = [(soma, apical, 10*nS), (soma, basal, 10*nS)]
pyr_model = NeuronModel(edges, cm=1*uF/(cm**2), gl=50*uS/(cm**2),
                          v_rest=-70*mV, r_axial=150*ohm*cm,
                          scale_factor=3, spine_factor=1.5)

# create a Brian NeuronGroup and link it to the neuron model
pyr_group = b.NeuronGroup(3, model=pyr_model.equations, method='euler',
                          threshold='V_soma > -40*mV', reset='V_soma = -50*mV',
                          refractory=3*ms, namespace=pyr_model.parameters)
pyr_model.link(pyr_group)

***We can inspect what Dendrify created for us simply by printing a NeuronModel object:***

In [ ]:
print(pyr_model)

***Also we can access any of its properties using the dot notation (example:):***


In [ ]:
pyr_model.parameters

***Let's replicate now the experiments of Figure 2:***

In [ ]:
# Set monitors for variables of interest
M = b.StateMonitor(pyr_group, ["V_soma", "V_apical", "V_basal"], record=True)

# Run simulation
I = 100 * pA
b.run(100*ms)
pyr_group.I_ext_soma[0] = I
pyr_group.I_ext_apical[1] = I
pyr_group.I_ext_basal[2] = I
b.run(400*ms)
pyr_group.I_ext_soma[0] = 0 * pA
pyr_group.I_ext_apical[1] = 0 * pA
pyr_group.I_ext_basal[2] = 0 * pA
b.run(550*ms)

In [ ]:
time = M.t/ms
vs = M.V_soma/mV
va = M.V_apical/mV
vb = M.V_basal/mV

fig, axes = plt.subplots(1, 3, figsize=[10, 3], sharex=True, sharey=True)
ax0, ax1, ax2 = axes

for i in range(3):
  ax0.plot(time, vs[i])
  ax1.plot(time, vs[i])
  ax2.plot(time, vs[i])


plt.show()

In [ ]:
vs = M.V_soma/mV

In [ ]:
vs